# As principais palavras mais comuns em Drácula, por Bram Stoker

Considerado como um marco da literatura gótica, o icônico livro Drácula, escrito em 1897 por Bram Stoker, desperta até hoje o fascínio das pessoas por todo o mundo. A fim de consolidar os conhecimentos iniciais do Apache Spark, desenvolveu-se este notebook para analisar as principais palavras mais comuns encontradas neste clássico livro.

A obra em questão foi obtida por meio do [Projeto Gutenberg]( https://www.gutenberg.org/), um acervo digital que reúne livros de todo o mundo que já se encontram em domínio público. A versão plaintext de Drácula pode ser baixada gratuitamente [aqui]( https://www.gutenberg.org/cache/epub/345/pg345.txt).

O presente notebook consiste nas seguintes etapas, explanados com mais detalhes no decorrer do desenvolvimento:

1. Download do livro Drácula, por Bram Stoker;
2. Inicialização do Apache Spark e leitura do livro;
3. Extração individual das palavras em cada uma das linhas;
4. Explodindo a lista de palavras em colunas no DataFrame;
5. Transformando todas as palavras em minúsculas;
6. Eliminação de pontuação;
7. Remoção de valores nulos;
8. Frequência das palavras mais comuns;


## Passo um: Download do livro

O download do livro consiste na solicitação da URL onde se encontra o arquivo TXT através da biblioteca requests. Depois, salva-se o conteúdo da solicitação, isto é, o próprio livro, no diretório atual, com o nome de **Dracula – Bram Stoker.txt**.


In [ ]:
import requests


url = "https://www.gutenberg.org/cache/epub/345/pg345.txt"
filename = "Dracula - Bram Stoker.txt"

r = requests.get(url)

with open(filename, "wb") as f:
    f.write(r.content)


## Passo dois: inicialização do Apache Spark e leitura do livro;

A seguir, cria-se uma instância da classe SparkSession, a qual irá realizar a inicialização do Apache Spark.

Depois, é feito a leitura do arquivo TXT previamente baixado com o Spark.


In [42]:
from pyspark.sql import SparkSession


spark = (SparkSession.builder
         .appName("The top most common words in Dracula, by Bram Stoker")
         .getOrCreate()
         )

sc = spark.sparkContext


22/07/12 11:29:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [43]:
book = spark.read.text(filename)


## Passo três: Extração individual das palavras em cada uma das linhas;

Após a leitura do livro ser concluída, é necessário que transforme cada uma das palavras em uma coluna no DataFrame.

Para isso, utiliza-se o método **split**, o qual, para cada uma das linhas, irá separar cada uma das palavras através do espaço em branco entre elas. O resultado será uma lista de palavras.


In [44]:
from pyspark.sql.functions import split


lines = book.select(split(book.value, " ").alias("line"))
lines.show(5)


+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[most, other, par...|
|[whatsoever., You...|
+--------------------+
only showing top 5 rows



## Passo quatro: explodindo a lista de palavras em colunas no DataFrame;

Depois das palavras terem sido separadas, é necessário que se faça a conversão desta lista de palavras em colunas no DataFrame.

Para tal, usa-se o método **explode** presente no Apache Spark.


In [45]:
from pyspark.sql.functions import explode, col


words = lines.select(explode(col("line")).alias("word"))
words.show(15)


+---------+
|     word|
+---------+
|      The|
|  Project|
|Gutenberg|
|    eBook|
|       of|
| Dracula,|
|       by|
|     Bram|
|   Stoker|
|         |
|     This|
|    eBook|
|       is|
|      for|
|      the|
+---------+
only showing top 15 rows



## Passo cinco: transformando todas as palavras em minúsculas;

Para que não haja distinção da mesma palavra por conta de letras maiúsculas, transforma-se todas as palavras no DataFrame para letras minúsculas, fazendo o uso da função **lower**.


In [46]:
from pyspark.sql.functions import lower


words_lower = words.select(lower(col("word")).alias("word_lower"))
words_lower.show()


+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|  dracula,|
|        by|
|      bram|
|    stoker|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
|  anywhere|
|        in|
+----------+
only showing top 20 rows



## Passo seis: eliminação de pontuação;

Para que também não haja distinção da mesma palavra por conta da pontuação presente no final delas, é preciso removê-las.

Isso é feito através do método **regexp_extract***, o qual extrai palavras de uma string por meio de uma expressão regular. Neste caso, a expressão regular em questão consiste em um conjunto contendo todos os símbolos de A a Z, uma ou mais vezes.


In [47]:
from pyspark.sql.functions import regexp_extract


words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]+", 0).alias("word")
)

words_clean.show()


+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|  dracula|
|       by|
|     bram|
|   stoker|
|         |
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
|       in|
+---------+
only showing top 20 rows



## Passo sete:  remoção de valores nulos;

Como visto, mesmo após a remoção das pontuações ainda há colunas com valores nulos, ou seja, espaços em branco.

Para que esses espaços em branco não sejam considerados na análise da frequência de cada palavra presente no livro, é necessário removê-los.


In [48]:
words_nonull = words_clean.filter(col("word") != "")
words_nonull.show()


+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|  dracula|
|       by|
|     bram|
|   stoker|
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
|       in|
|      the|
+---------+
only showing top 20 rows



## Passo oito: frequência das palavras mais comuns

E, por fim, é realizado a contagem das palavras mais comuns presentes no livro. Para isso, agrupa-se cada uma das palavras e depois usa-se uma função de agregação, **count**, para determinar quantas vezes elas aparecem.

Depois, exibe as 100 palavras mais comuns. O ranque pode ser ajustado através da variável **rank**.


In [49]:
words_count = (words_nonull.groupby(col("word"))
               .count()
               .orderBy(col("count"), ascending=False)
               )


In [50]:
rank = 100
words_count.show(rank)


+-------+-----+
|   word|count|
+-------+-----+
|    the| 8046|
|    and| 5897|
|      i| 4760|
|     to| 4733|
|     of| 3743|
|      a| 2990|
|     in| 2561|
|     he| 2558|
|   that| 2475|
|     it| 2172|
|    was| 1878|
|     as| 1582|
|     we| 1545|
|    for| 1534|
|     is| 1523|
|    you| 1479|
|    his| 1469|
|     me| 1454|
|    not| 1420|
|   with| 1321|
|     my| 1259|
|    all| 1172|
|     be| 1133|
|     so| 1095|
|     at| 1091|
|     on| 1082|
|    but| 1069|
|   have| 1065|
|    her| 1060|
|    had| 1038|
|    him|  957|
|    she|  814|
|  there|  775|
|   when|  772|
|   this|  667|
|  which|  659|
|     if|  649|
|   from|  637|
|    are|  605|
|   said|  569|
|   were|  551|
|   then|  547|
|     by|  541|
|     or|  526|
|    one|  505|
|  could|  493|
|     do|  492|
|     no|  485|
|   them|  472|
|   they|  466|
|   what|  465|
|     us|  463|
|   will|  460|
|   must|  451|
|     up|  447|
|   some|  441|
|  would|  430|
|    out|  430|
|    may|  428|
|  shall

## Referências
RIOUX, Jonathan. [Data Analysis with Python and PySpark](https://www.amazon.com.br/Analysis-Python-PySpark-Jonathan-Rioux/dp/1617297208).

STOKER, Bram. [Dracula](https://www.gutenberg.org/cache/epub/345/pg345.txt).
